In [1]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))

226618


In [11]:
chars = sorted(set(text))
vocab_size = len(chars)

In [12]:
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for ch,i in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: [int_to_string[s] for s in l]

In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.size())
print(data[:100])

torch.Size([226618])
tensor([79,  0, 26, 31, 24, 39, 43, 28, 41,  1, 12, 10,  0,  0, 43, 31, 28,  1,
        28, 24, 41, 43, 31, 40, 44, 24, 34, 28,  0,  0,  0, 43, 60, 57,  1, 72,
        70, 53, 61, 66,  1, 58, 70, 67, 65,  1,  5, 29, 70, 61, 71, 55, 67,  1,
        75, 53, 71,  1, 74, 57, 70, 77,  1, 64, 53, 72, 57, 10,  1, 32, 72,  1,
        71, 60, 67, 73, 64, 56,  1, 60, 53, 74, 57,  1, 53, 70, 70, 61, 74, 57,
        56,  1, 53, 72,  1, 31, 73, 59, 71, 67])


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

block_size=8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]

    print(f"when context is {context}, target is {target}")

when context is tensor([79]), target is 0
when context is tensor([79,  0]), target is 26
when context is tensor([79,  0, 26]), target is 31
when context is tensor([79,  0, 26, 31]), target is 24
when context is tensor([79,  0, 26, 31, 24]), target is 39
when context is tensor([79,  0, 26, 31, 24, 39]), target is 43
when context is tensor([79,  0, 26, 31, 24, 39, 43]), target is 28
when context is tensor([79,  0, 26, 31, 24, 39, 43, 28]), target is 41


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [8]:
batch_size = 4

In [10]:
def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size + 1, (batch_size, ))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print("inputs:")
print(x)
print("targets:")
print(y)

tensor([ 53172, 136500, 138151,  71106])


D:\Users\Danya\LLM\cuda\Lib\site-packages\torch\cuda\__init__.py:262: UserWarning: 
    Found GPU0 GeForce GT 730 which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(


inputs:
tensor([[ 1, 75, 53, 71,  1, 71, 72, 61],
        [72, 70, 61, 68,  8,  1, 58, 67],
        [64, 57, 58, 72,  1, 61, 66,  1],
        [66, 72,  1, 75, 60, 57, 70, 57]], device='cuda:0')
targets:
tensor([[75, 53, 71,  1, 71, 72, 61, 64],
        [70, 61, 68,  8,  1, 58, 67, 70],
        [57, 58, 72,  1, 61, 66,  1, 60],
        [72,  1, 75, 60, 57, 70, 57,  1]], device='cuda:0')


In [ ]:
import torch.nn as nn
from torch.nn import functional as F
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets):
        logits = self.token_embedding_table(index)

        return logits